# Observed Occupations-Task Dataset

By Paul Duckworth 11th Dec 2017.

Edit the Oberved dataset, to insert additional occupation/tasks combo's using the "also done by columns". 

## Observational Data:

In [71]:
# encoding=utf8
import os
import numpy as np
import pandas as pd
import getpass
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

datasets = '/home/'+ getpass.getuser() +'/Datasets/'
print(datasets)
d = os.path.join(datasets, 'FoHealthcare/Tasks_dataset_170618.xlsx')
excel_doc = pd.ExcelFile(d)
matts_data = excel_doc.parse()
print(matts_data.shape)

# matts_data = excel_doc.parse("Title, Tasks, Features").fillna("-")
# d = Tasks_dataset_170618'FoHealthcare/tasks_apr18.csv')
# matts_data = pd.read_csv(d)
matts_data.rename(columns = {'Occupation title':'Observed Occupation'}, inplace = True)

columns_required = ["Observed Occupation", "Task", "DWA ID", "Task Weight", "Task also done by"]
data = matts_data#[columns_required]

unique_occupations = [i.strip().lower() for i in data["Observed Occupation"].unique()]+["scanning clerk"]
data

/home/scpd/Datasets/
(148, 19)


,Observed Occupation,Task,Task keywords/context,GP Code,DWA Task,DWA ID,Task Weight,Notes,Task also done by,Technology Use,Technology use3,Technology use4,Information work,Information work5,Information work6,Information work7,partial automation,Future automation potential (FAP),FAP Notes
0,Administrator,Medical Coding,Structured clinical vocabulary highlight docum...,"BFS, BSC, WMC",Code data or other information.; Convert data ...,4.A.3.b.1.I06.D08; 4.A.3.b.1.I06.D11; 4.A.3.b....,NaN,NaN,Summariser; deputy practice manager; Administr...,Desktop,specific software,NaN,reduction,NaN,NaN,NaN,no,NaN,NaN
1,Administrator,Register new Patients,NaN,"BFS, BSC",Process healthcare paperwork.; Verify accuracy...,4.A.4.c.1.I01.D03; 4.A.2.a.2.I05.D04; 4.A.3.b....,NaN,Summarise patient records that have just moved...,Receptionist; Summariser,Desktop,paper,NaN,transfer,entry,structured,error checking,no,NaN,NaN
2,Administrator,Use Intellisense to OCR letters and pick out c...,NaN,BFS,Convert data among multiple digital or analog ...,4.A.3.b.1.I06.D11; 4.A.2.a.3.I01.D02,NaN,NaN,NaN,Desktop,software,NaN,reduction,NaN,NaN,NaN,yes,NaN,NaN
3,Administrator,Child immunization targets in open exeter,NaN,WMC,Monitor external affairs or events affecting b...,4.A.1.a.2.I08.D03; 4.A.4.c.1.I01.D03,NaN,NaN,NaN,Desktop,website,NaN,transfer,NaN,NaN,NaN,yes,NaN,NaN
4,Administrator,Letter writing,NaN,MWMC,Type documents.; Edit written materials.; Read...,4.A.4.c.1.I01.D07; 4.A.2.b.1.I07.D01; 4.A.1.a....,NaN,"Letters written as needed, changes day to day ...",Receptionist; Secretary; General Practitioner,Desktop,software,NaN,transfer,bricolage,NaN,NaN,yes,NaN,NaN
5,Administrator,Provide data to GPs from pallative care meetings.,NaN,WMC,Relay information between personnel. ; Send in...,4.A.4.a.2.I03.D11; 4.A.4.c.1.I05.D03,NaN,NaN,NaN,Desktop,NaN,NaN,social transaction,transfer,NaN,NaN,no,NaN,NaN
6,Administrator,Enter data for enhanced services,NaN,WMC,Develop procedures for data entry or processin...,4.A.2.b.4.I03.D16; 4.A.3.b.6.I10.D02,NaN,NaN,NaN,Desktop,website,NaN,transfer,social transaction,NaN,NaN,yes,NaN,NaN
7,Administrator,"Patient recalls for QOF, Write letters or text...",write quality outcomes framework letter on tex...,WMC,Search information sources to find specific da...,4.A.1.a.1.I04.D03; 4.A.4.c.1.I01.D07; 4.A.2.b....,NaN,NaN,deputy practice manager,Desktop,software,NaN,transfer,NaN,NaN,NaN,yes,NaN,NaN
8,Administrator,Process prescription declaration forms,NaN,WMC,Process healthcare paperwork.,4.A.4.c.1.I01.D03,NaN,NaN,NaN,Desktop,NaN,NaN,structured,error checking,NaN,NaN,no,NaN,NaN
9,Administrator,"Audit forms, auditing and keeping track of dat...",NaN,WMC,Process healthcare paperwork.,4.A.4.c.1.I01.D03,NaN,NaN,NaN,Desktop,NaN,NaN,structured,bricolage,unstructured,NaN,no,NaN,NaN


## Get Automation Labels:

In [63]:
# onet_datasets = '/home/'+ getpass.getuser() +'/Datasets/ONET/db2016'

y_train = pd.read_csv("icml_data/y_train.csv")
y_test = pd.read_csv("icml_data/y_test.csv")

y_train.rename(columns={"GT Score 1-4":"y"}, inplace=True)
y_test.rename(columns={"y_pred": "y"}, inplace=True)

automation_scores = pd.concat([y_train, y_test], ignore_index=True).iloc[:, [1,2]]
print(automation_scores.shape)

# test:
automation_scores[automation_scores["DWA ID"] == "4.A.3.b.1.I06.D11"]["y"].values[0]

(2067, 2)


2.90894657417

## Add Task Weights and DWA Scores:

In [64]:
data["Automation Scores"] =  data.apply(lambda _: '', axis=1)
data["Average Automation Score"] =  data.apply(lambda _: '', axis=1)
# data = data.reindex(columns = columns_required + ["Automation Scores", "Average Automation Score"])    

for i, (occ, task, str_dwas) in data[["Observed Occupation", "Task", "DWA ID"]].iterrows():
    
    # # Compute the weight (based on number of DWAs selected to Match)
    dwas = str_dwas.split(";")
    weight = 1./len(dwas)
    
    # # Grab the Automation Scores per DWA ID
    str_auto_scores = ""
    flo_auto_scores = []
    for d in dwas:
        try:
            float_y = automation_scores[automation_scores["DWA ID"] == d.strip()]["y"].values[0]
            y = " %0.2f;" % float_y
        except:
            float_y = 0
            y = ""
            
        # # Average the Automation scores to compute the Observed Task's Automation Score
        flo_auto_scores.append(float_y)
        str_auto_scores += y
#     print(dwas, str_auto_scores, flo_auto_scores, np.mean(flo_auto_scores))

    # # Put everything into the DataFrame
    data.iloc[i]["Task Weight"] = " %0.2f;" %weight*len(dwas)
    data.iloc[i]["Automation Scores"] = str_auto_scores
    data.iloc[i]["Average Automation Score"] = np.mean(flo_auto_scores)
data

,Observed Occupation,Task,Task keywords/context,GP Code,DWA Task,DWA ID,Task Weight,Notes,Task also done by,Technology Use,...,Technology use4,Information work,Information work5,Information work6,Information work7,partial automation,Future automation potential (FAP),FAP Notes,Automation Scores,Average Automation Score
0,Administrator,Medical Coding,Structured clinical vocabulary highlight docum...,"BFS, BSC, WMC",Code data or other information.; Convert data ...,4.A.3.b.1.I06.D08; 4.A.3.b.1.I06.D11; 4.A.3.b....,0.25; 0.25; 0.25; 0.25;,NaN,Summariser; deputy practice manager; Administr...,Desktop,...,NaN,reduction,NaN,NaN,NaN,no,NaN,NaN,3.08; 2.91; 3.50; 2.94;,3.10935
1,Administrator,Register new Patients,NaN,"BFS, BSC",Process healthcare paperwork.; Verify accuracy...,4.A.4.c.1.I01.D03; 4.A.2.a.2.I05.D04; 4.A.3.b....,0.33; 0.33; 0.33;,Summarise patient records that have just moved...,Receptionist; Summariser,Desktop,...,NaN,transfer,entry,structured,error checking,no,NaN,NaN,2.91; 3.15; 3.50;,3.18928
2,Administrator,Use Intellisense to OCR letters and pick out c...,NaN,BFS,Convert data among multiple digital or analog ...,4.A.3.b.1.I06.D11; 4.A.2.a.3.I01.D02,0.50; 0.50;,NaN,NaN,Desktop,...,NaN,reduction,NaN,NaN,NaN,yes,NaN,NaN,2.91; 2.94;,2.92687
3,Administrator,Child immunization targets in open exeter,NaN,WMC,Monitor external affairs or events affecting b...,4.A.1.a.2.I08.D03; 4.A.4.c.1.I01.D03,0.50; 0.50;,NaN,NaN,Desktop,...,NaN,transfer,NaN,NaN,NaN,yes,NaN,NaN,2.20; 2.91;,2.55488
4,Administrator,Letter writing,NaN,MWMC,Type documents.; Edit written materials.; Read...,4.A.4.c.1.I01.D07; 4.A.2.b.1.I07.D01; 4.A.1.a....,0.20; 0.20; 0.20; 0.20; 0.20;,"Letters written as needed, changes day to day ...",Receptionist; Secretary; General Practitioner,Desktop,...,NaN,transfer,bricolage,NaN,NaN,yes,NaN,NaN,3.50; 2.42; 3.32; 3.40; 2.57;,3.04252
5,Administrator,Provide data to GPs from pallative care meetings.,NaN,WMC,Relay information between personnel. ; Send in...,4.A.4.a.2.I03.D11; 4.A.4.c.1.I05.D03,0.50; 0.50;,NaN,NaN,Desktop,...,NaN,social transaction,transfer,NaN,NaN,no,NaN,NaN,3.38; 3.40;,3.38665
6,Administrator,Enter data for enhanced services,NaN,WMC,Develop procedures for data entry or processin...,4.A.2.b.4.I03.D16; 4.A.3.b.6.I10.D02,0.50; 0.50;,NaN,NaN,Desktop,...,NaN,transfer,social transaction,NaN,NaN,yes,NaN,NaN,2.70; 3.46;,3.08005
7,Administrator,"Patient recalls for QOF, Write letters or text...",write quality outcomes framework letter on tex...,WMC,Search information sources to find specific da...,4.A.1.a.1.I04.D03; 4.A.4.c.1.I01.D07; 4.A.2.b....,0.25; 0.25; 0.25; 0.25;,NaN,deputy practice manager,Desktop,...,NaN,transfer,NaN,NaN,NaN,yes,NaN,NaN,2.57; 3.50; 2.42; 3.42;,2.97943
8,Administrator,Process prescription declaration forms,NaN,WMC,Process healthcare paperwork.,4.A.4.c.1.I01.D03,1.00;,NaN,NaN,Desktop,...,NaN,structured,error checking,NaN,NaN,no,NaN,NaN,2.91;,2.91399
9,Administrator,"Audit forms, auditing and keeping track of dat...",NaN,WMC,Process healthcare paperwork.,4.A.4.c.1.I01.D03,1.00;,NaN,NaN,Desktop,...,NaN,structured,bricolage,unstructured,NaN,no,NaN,NaN,2.91;,2.91399


## Also Done By: add extra rows

In [65]:
unique_occupations

['administrator',
 'deputy practice manager',
 'general practitioner',
 'healthcare assistant',
 'nurse practitioner',
 'pharmacy technician',
 'phlebotomist',
 'practice manager',
 'practice nurse',
 'practice pharmacist',
 'prescription clerk',
 'receptionist',
 'secretary',
 'summariser']

In [72]:
data.shape
duplicate_task_rows = data.copy()
for i, row in duplicate_task_rows.iterrows():
#     print(i, row['Observed Occupation'], row['Task'])
    try:
        occu = row['Observed Occupation'].strip().lower()
        also_done_by = row['Task also done by'].split(";")
        for also_occ in also_done_by:
            a = also_occ.strip().lower()
            if a == occu: continue
            elif a in unique_occupations: 
                row['Observed Occupation'] = a
                duplicate_task_rows = duplicate_task_rows.append(row, ignore_index=True)
#             elif a in "all staff":
#                 for all_occs in unique_occupations:
#                     row['Observed Occupation'] = all_occs
#                     duplicate_task_rows = duplicate_task_rows.append(row, ignore_index=True)
            else:
                print("***", i, "different occupation:", a)
            
    except AttributeError:  # some columns contain NaN, i.e. a "float", that can't be stripped. 
        pass
print(duplicate_task_rows.shape)
duplicate_task_rows[duplicate_task_rows["Task"] == "Admin"]
# duplicate_task_rows


*** 26 different occupation: 
*** 49 different occupation: 
*** 110 different occupation: all staff
(277, 19)


,Observed Occupation,Task,Task keywords/context,GP Code,DWA Task,DWA ID,Task Weight,Notes,Task also done by,Technology Use,Technology use3,Technology use4,Information work,Information work5,Information work6,Information work7,partial automation,Future automation potential (FAP),FAP Notes
26,Deputy Practice Manager,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN
166,general practitioner,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN
167,healthcare assistant,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN
168,nurse practitioner,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN
169,practice nurse,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN
170,secretary,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN
171,administrator,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN
172,receptionist,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN
173,practice manager,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN
174,pharmacy technician,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,NaN,2-3 hours a day for some practice staff,General Practitioner; Healthcare Assistant; Nu...,Desktop,NaN,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN


In [33]:
duplicate_task_rows.sort_values(by = ['Observed Occupation', 'Task'], inplace=True)
# duplicate_task_rows.drop(['Task also done by'], axis=1, inplace=True)
duplicate_task_rows.reset_index()

,index,Observed Occupation,Task,Task keywords/context,GP Code,DWA Task,DWA ID,Task Weight,Notes,Task also done by,...,Technology use4,Information work,Information work5,Information work6,Information work7,partial automation,Future automation potential (FAP),FAP Notes,Automation Scores,Average Automation Score
0,174,,Admin,NaN,BFS,Perform administrative or clerical tasks.,4.A.4.c.1.I01.D10,1.00;,2-3 hours a day for some practice staff,"General Practitioner; Healthcare Assistant, Nu...",...,NaN,unstructured,bricolage,entry,NaN,no,NaN,NaN,2.65;,2.64976
1,185,,Cleaning workspace after every patient,NaN,WMC,Clean work areas.; Clean objects.; Clean medic...,4.A.3.a.1.I03.D05; 4.A.3.a.1.I08.D06; 4.A.3.a....,0.33; 0.33; 0.33;,takes a lot of time and lots of work. Same pro...,Nurse Practitioner; General Practitioner; Phle...,...,NaN,NaN,NaN,NaN,NaN,no,yes,robotic solutions for cleaning counters and wo...,3.00; 2.95; 2.91;,2.95093
2,9,Administrator,"Audit forms, auditing and keeping track of dat...",NaN,WMC,Process healthcare paperwork.,4.A.4.c.1.I01.D03,1.00;,NaN,NaN,...,NaN,structured,bricolage,unstructured,NaN,no,NaN,NaN,2.91;,2.91399
3,3,Administrator,Child immunization targets in open exeter,NaN,WMC,Monitor external affairs or events affecting b...,4.A.1.a.2.I08.D03; 4.A.4.c.1.I01.D03,0.50; 0.50;,NaN,NaN,...,NaN,transfer,NaN,NaN,NaN,yes,NaN,NaN,2.20; 2.91;,2.55488
4,6,Administrator,Enter data for enhanced services,NaN,WMC,Develop procedures for data entry or processin...,4.A.2.b.4.I03.D16; 4.A.3.b.6.I10.D02,0.50; 0.50;,NaN,NaN,...,NaN,transfer,social transaction,NaN,NaN,yes,NaN,NaN,2.70; 3.46;,3.08005
5,13,Administrator,Invoicing (private insurance),NaN,NaN,Process customer bills or payments.,4.A.4.c.1.I03.D06,1.00;,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.80;,3.8
6,4,Administrator,Letter writing,NaN,MWMC,Type documents.; Edit written materials.; Read...,4.A.4.c.1.I01.D07; 4.A.2.b.1.I07.D01; 4.A.1.a....,0.20; 0.20; 0.20; 0.20; 0.20;,"Letters written as needed, changes day to day ...",Receptionist; Secretary; General Practitioner,...,NaN,transfer,bricolage,NaN,NaN,yes,NaN,NaN,3.50; 2.42; 3.32; 3.40; 2.57;,3.04252
7,11,Administrator,Mass mail letters for checkups using DocMail,print spreadsheet post mail automated address ...,WMC,Enter information into databases or software p...,4.A.4.c.1.I05.D02; 4.A.3.b.1.I06.D06,0.50; 0.50;,NaN,NaN,...,NaN,transfer,structured,NaN,NaN,yes,NaN,NaN,3.42; 3.50;,3.46243
8,7,Administrator,"Patient recalls for QOF, Write letters or text...",write quality outcomes framework letter on tex...,WMC,Search information sources to find specific da...,4.A.1.a.1.I04.D03; 4.A.4.c.1.I01.D07; 4.A.2.b....,0.25; 0.25; 0.25; 0.25;,NaN,deputy practice manager,...,NaN,transfer,NaN,NaN,NaN,yes,NaN,NaN,2.57; 3.50; 2.42; 3.42;,2.97943
9,8,Administrator,Process prescription declaration forms,NaN,WMC,Process healthcare paperwork.,4.A.4.c.1.I01.D03,1.00;,NaN,NaN,...,NaN,structured,error checking,NaN,NaN,no,NaN,NaN,2.91;,2.91399


In [ ]:
# duplicate_task_rows.sort_values(by = ['Task', 'Observed Occupation'], inplace=True)
output_doc = os.path.join(datasets, 'FoHC/Duplicated_also_done_by_tasks.csv')
duplicate_task_rows.to_csv(output_doc, sep='\t', encoding='utf-8')
